# YouTube & Spotify Top Music Artists From 2018
---

### SCOPE:
#### - Extracted, transformed, and loaded up YouTube's Top Trending Videos from December 2017 thru May 2018 for their videos categorized as music only, and created an "Artist" column to enable joining with Spotify's Top 100 Songs of 2018. Both dataframes were loaded into MySQL.


### PURPOSE:
#### - I choose this project because I'm a avid listener and a huge music and concert goer, and wanted to work with data that I was familiar with.

### Data Sources:
#### - https://www.kaggle.com/datasnaek/youtube-new (this is an updated link, whereas I used an older version of this file, which is attached in the resources)
#### - https://www.kaggle.com/nadintamer/top-spotify-tracks-of-2018
 


In [ ]:
# Import Dependencies 1/2:
import os
import csv
import json
import simplejson
import numpy as np
import pandas as pd
from datetime import datetime
import sys
import string

## Part 1) SQL Alchemy setup and built a search, create, and drop database function to set up loading phase after extraction and transformation of data

In [ ]:
# Import Dependencies 2/2:
from sqlalchemy import create_engine, Column, Integer, String, join, Date, Table, MetaData
from sqlalchemy.sql import select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import database_exists, create_database, drop_database, has_index
import pymysql

Base = declarative_base()

In [ ]:
#rds_connection_string = "<inser user name>:<insert password>@127.0.0.1/customer_db"
rds_connection_string = "root:PASSWORD@127.0.0.1/" #youtube_spotify_2018_db"

# Can set up an input for the db_name later (optional)
#db_name = input("What database would you like to search for?")
db_name = 'youtube_spotify_2018_db2'

# Setup engine connection string
#engine = create_engine(f'mysql://{rds_connection_string}{db_name}?charset=utf8')
engine = create_engine(f'mysql://{rds_connection_string}{db_name}?charset=utf8', echo=True)

In [ ]:
# Created a function incorproating SQL Alchemy to search, create, and or drop a database:
def search_create_drop_db(db_name):
    #db_exist = database_exists(f'mysql://{rds_connection_string}youtube_spotify_2018_db2')
    db_exist = database_exists(f'mysql://{rds_connection_string}{db_name}')
    db_url = f'mysql://{rds_connection_string}{db_name}'
    if db_exist == True:
        drop_table_y_or_n = input(f'"{db_name}" database already exists in MySQL. Do you want you drop the table? Enter exactly: "y" or "n".  ')
        if drop_table_y_or_n == 'y':
            drop_database(db_url)
            print(f"Database {db_name} was dropped")
            create_new_db = input(f"Do you want to create another database called: {db_name}?  ")
            if create_new_db == 'y':
                create_database(db_url)
                return(f"The database {db_name} was created. Next You will need to create tables for this database.  ")
            else:
                return("No database was created. Goodbye!  ")
        else:
            return("The database exists. No action was taken. Goodbye!  ")
    else:
        create_database(db_url)
        return(f"The queried database did not exist, and was created as: {db_name} .  ")

search_create_drop_db(db_name)

In [ ]:
# # Create CLASSES / AKA tables 'blueprints' using python classes and sql alchemy:
# # This would be useful say if I was to use SQL Alchemy to import a sqllite file into python

# class yt_categories(Base):
#     __tablename__ = 'yt_category_titles'
#     id = Column(Integer, primary_key=True)
#     category_title = Column(String(60))

# class yt_statistics_data(Base):
#     __tablename__ = 'yt_statistics'
#     id = Column(Integer, primary_key=True)
#     category_title = Column(String(60))
#     trending_date = Column(Date, nullable=False)
#     video_title = Column(String(200))
#     channel_title = Column(String(100))
#     category_id = Column(Integer)
#     views = Column(Integer)
#     likes = Column(Integer)
#     dislikes = Column(Integer)
#     view_count = Column(Integer)

# class spotify_music_data(Base):
#     __tablename__ = 'spotify_music'
#     id = Column(Integer, primary_key=True)
#     artist = Column(String(100))
#     song_name = Column(String(200))
#     spotify_unique_id = Column(String(100))


In [ ]:
# Create tables 'blueprints' using python classes and sql alchemy:

meta = MetaData()
conn = engine.connect()

yt_categories = Table(
   'yt_categories', meta, 
   Column('category_id', Integer, primary_key = True), 
   Column('category_title', String)
)

yt_statistics = Table(
   'yt_statistics', meta,
   Column('id', Integer, primary_key = True),
   Column('trending_date', Date, nullable=False),
   Column('video_title', String),
   Column('channel_title', String),
   Column('category_id', Integer),
   Column('views', Integer),
   Column('likes', Integer),
   Column('dislikes', Integer),
   Column('comment_count', Integer)
)

spotify_2018top100_data = Table(
   'spotify_2018top100_data', meta,
   Column('Artist', String, primary_key = True),
   Column('song_name', String),
   Column('unique_id', String)
)



## Part 2a) ETL on the "YouTube Categories" JSON dataset:
### Extraction, transformation, load, and read using 2 methods from MySQL Database 

In [ ]:
# YouTube data has two parts: 1) Categories information in JSON format
                            # 2) Top Trending US YouTube Videos in a CSV file

# Part 1) YouTube Categories are seperated in a json file
yt_json_file = './resources/youtube_US_category_id.json'
yt_rawjson_df = pd.read_json(yt_json_file)

In [ ]:
# Extract the category id and category titles, and set them into a list

# for i in yt_rawjson_df['items']:
#     #print(i['id'])
#     print(i['id'] + ' | ' + i['snippet']['title'])
    

category_id = [int(i['id']) for i in yt_rawjson_df['items']]
category_title = [str(i['snippet']['title']) for i in yt_rawjson_df['items']]

# Create a dataframe of the category id and title for later use
category_id_title_df = pd.DataFrame({'category_id': category_id, 'category_title': category_title})
category_id_title_df.head()
category_id_title_df.info()

In [ ]:
# Drop Dulplicates and Sort by category_title
category_id_title_df.drop_duplicates(['category_id', 'category_title']).sort_values(by=['category_title'], ascending=False).head()

In [ ]:
# Load category_id_title_df to MySQL with Pandas
category_id_title_df.to_sql('yt_categories', con=engine, if_exists='append')

In [ ]:
# Two ways in this notebook to pull the data directly from MySQL database
# Method 1) Use SQL Alchemy Engine - Result: successfully reads from MySQL Database:
engine.execute("SELECT * FROM yt_categories").fetchall()

In [ ]:
# Method 2) Read from MySQL database using Pandas - Result: Success! - This method is a better setup for analysis.
# The index column is automatically generated.
yt_cat_df = pd.read_sql_query('SELECT * FROM yt_categories', con=engine)
yt_cat_df.head()

## Part 2b) ETL on the "YouTube Top Trending US Videos" dataset:
### Extraction, transformation, load, and read using 2 methods from MySQL Database 

In [ ]:
# Part 1) is the YouTube Top US Videos in a CSV
csv_file_yt = "./resources/youtube_USvideos.csv"
yt_rawdata_df = pd.read_csv(csv_file_yt, encoding='utf-8')

In [ ]:
# view rows, count and datatypes
yt_rawdata_df
yt_rawdata_df['id'] = yt_rawdata_df.index
yt_rawdata_df.head(1)

In [ ]:
# Rename Columns
yt_statistics_data_df = yt_rawdata_df.rename(columns={
                                                "title":"video_title",
                                                "category_id":"category_id"
                                               })
yt_statistics_data_df.head()

In [ ]:
# Drop Cells with Missing Information
yt_statistics_data_df = yt_statistics_data_df.dropna(how="any")

In [ ]:
# Drop Dulplicates and Sort by Trending Date
yt_statistics_data_df.drop_duplicates(['video_id', 'trending_date', 'video_title', 'channel_title', 'category_id', 'publish_time', 'views', 'likes', 'dislikes', 'comment_count']).sort_values(by=['trending_date'], ascending=False).head()

In [ ]:
# Drop Unwanted Columns

to_drop =['video_id', 'publish_time', 'tags', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description']

yt_statistics_data_df.drop(to_drop, inplace=True, axis=1)
yt_statistics_data_df.head(1)

In [ ]:
# Reorganize columns to match SQL Alchemy Table

cols = yt_statistics_data_df.columns.tolist()
cols = cols[-1:] + cols[0:-1]
cols
yt_statistics_data_df = yt_statistics_data_df[cols]

In [ ]:
# Replace the "." in Trending Date to "-"

yt_statistics_data_df['trending_date'] = [x.replace(".","-") for x in yt_statistics_data_df['trending_date']]
yt_statistics_data_df.head()

In [ ]:
# Pulled up this dataframe verify exactly how "Music" was spelled for the following step:
category_id_title_df['category_title'].value_counts().head()

In [ ]:
# Used Pandas Merge aka (Join in Relational DB) Inner joined the Both YouTube Tables to find what Category_id 10 means:

yt_merged_df = pd.merge(yt_statistics_data_df, category_id_title_df, how='inner', on='category_id',
         left_index=False, right_index=False, sort=False)

yt_musicdata_df = yt_merged_df[yt_merged_df['category_title']  == 'Music']

In [ ]:
# Only going to clean and focus on YT videos that are in the "Music" category_title:

# Clean Channel Title Column to set up the MAIN LOOP

yt_musicdata_df['channel_title'] = [x.replace("VEVO","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("vevo","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("Vevo","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("Official","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("official","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("OFFICIAL","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("You Tube Channel","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("Music","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace("music","") for x in yt_musicdata_df['channel_title']]
yt_musicdata_df['channel_title'] = [x.replace(" - Topic","") for x in yt_musicdata_df['channel_title']]

yt_musicdata_df['channel_title'].replace("BackstreetBoys","Backstreet Boys")
yt_musicdata_df['channel_title'].replace("CalumScott","Calum Scott")
yt_musicdata_df['channel_title'].replace("TaylorSwift","Taylor Swift")
yt_musicdata_df['channel_title'].replace("NickiMinajAt","Nicki Minaj")
yt_musicdata_df['channel_title'].replace("FifthHarmony","FifthHarmony")
yt_musicdata_df['channel_title'].replace("davematthewsband","Dave Matthews Band")
yt_musicdata_df['channel_title'].replace("EnriqueIglesias","Enrique Iglesias")
yt_musicdata_df['channel_title'].replace("ChildishGambino","Childish Gambino")
yt_musicdata_df['channel_title'].replace("SamSmithWorld","Sam Smith")
yt_musicdata_df['channel_title'].replace("MeghanTrainor","Meghan Trainor")
yt_musicdata_df['channel_title'].replace("johnmayer","John Mayer")
yt_musicdata_df['channel_title'].replace("weezer","Weezer")
yt_musicdata_df['channel_title'].replace("AzealiaBanks","Azealia Banks")
yt_musicdata_df['channel_title'].replace("Maroon5","Maroon 5")
yt_musicdata_df['channel_title'].replace("Zayn","ZAYN")
yt_musicdata_df['channel_title'].replace("ArianaGrande","Ariana Grande")
yt_musicdata_df['channel_title'].replace("CAguilera","Christina Aguilera")
yt_musicdata_df['channel_title'].replace("LadyGaga","Lady Gaga")
yt_musicdata_df['channel_title'].replace("ToniBraxton","Toni Braxton")
yt_musicdata_df['channel_title'].replace("JasonAldean","Jason Aldean")
yt_musicdata_df['channel_title'].replace("PTXofficial","PTX")
yt_musicdata_df['channel_title'].replace("KeithUrban","Keith Urban")
yt_musicdata_df['channel_title'].replace("KaceyMusgraves","Kacey Musgraves")
yt_musicdata_df['channel_title'].replace("ChrisStapleton","Chris Stapleton")
yt_musicdata_df['channel_title'].replace("ThirtySecondsToMars","Thirty Seconds To Mars")

# Clean Special Characters to prevent latin-1 encoding errors. Went back up to the pd.read_csv
# and added "encoding="utf-8"

yt_musicdata_df['video_title'] = [x.replace("é","e") for x in yt_musicdata_df['video_title']]
yt_musicdata_df['video_title'] = [x.replace("ú","u") for x in yt_musicdata_df['video_title']]
yt_musicdata_df['video_title'] = [x.replace("®","") for x in yt_musicdata_df['video_title']]


In [ ]:
yt_musicdata_df.info()

In [ ]:
# Load category_id_title_df to MySQL with Pandas
yt_musicdata_df.to_sql('yt_statistics', con=engine, if_exists='append')

In [ ]:
# Two ways in this notebook to pull the data directly from MySQL database
# Method 1) Use SQL Alchemy Engine - Result: successfully reads from MySQL Database:
engine.execute("SELECT * FROM yt_statistics").fetchall()

In [ ]:
# Method 2) Read from MySQL database using Pandas - Result: Success! - This method is a better setup for analysis.
# The index column is automatically generated.
yt_stat_df = pd.read_sql_query('SELECT * FROM yt_statistics', con=engine)
yt_stat_df.head()

## Part 2c) ETL on the "Spotify Top 2018 Songs" dataset:
### Extraction, transformation, load, and read using 2 methods from MySQL Database 

In [ ]:
# Spotify 2018 - Top 100 Songs - Raw CSV

csv_file_spotify2018 = "./resources/spotify_top2018.csv"
spotify2018_rawdata_df = pd.read_csv(csv_file_spotify2018)
spotify2018_rawdata_df.head()

In [ ]:
spotify2018_rawdata_df.info()

In [ ]:
# Set up Spotify DataFrame
spotify_2018_id = spotify2018_rawdata_df['id']
spotify_2018_name = spotify2018_rawdata_df['name']
spotify_2018_artists = spotify2018_rawdata_df['artists']

In [ ]:
spotify2018_filtered_df = pd.DataFrame({
            "artists": spotify_2018_artists,
            "song_name": spotify_2018_name,
            "spotify_unique_id": spotify_2018_id
             })
spotify2018_filtered_df.head()

In [ ]:
# Load category_id_title_df to MySQL with Pandas
spotify2018_filtered_df.to_sql('spotify_2018top100_data', con=engine, if_exists='append')

In [ ]:
# Two ways in this notebook to pull the data directly from MySQL database
# Method 1) Use SQL Alchemy Engine - Result: successfully reads from MySQL Database:
engine.execute("SELECT * FROM spotify_2018top100_data").fetchall()

In [ ]:
# Method 2) Read from MySQL database using Pandas - Result: Success! - This method is a better setup for analysis.
# The index column is automatically generated.
spotify_df = pd.read_sql_query('SELECT * FROM spotify_2018top100_data', con=engine)
spotify_df.head()

## Part 3) Investigating how many of Spotify Top Artists that also had Top Trending Videos on YouTube in 2018

In [ ]:
# SQL Alchemy
j = yt_statistics.join(yt_categories, yt_statistics.c.category_id == yt_categories.c.category_id)
stmt = select([yt_statistics]).select_from(j)
result = conn.execute(stmt)
result.fetchall()

In [ ]:
yt_merged2_df = pd.merge(yt_stat_df, yt_cat_df, how='inner', on='category_id',
         left_index=False, right_index=False, sort=False)
yt_merged2_df.insert(4, "artist", '')

In [ ]:
# Nested loop to identify which artists from Spotify's Top 100 also had Top Trending Videos in 2018:
# Loop through both YouTube and Spotify Data Sets, Normalize them (lower case and remove spaces),
# and fill in the newly "Artist" column in the YouTube DF with the Spotify Artist Value if the artist name is found

for index, x in yt_merged2_df.iterrows():
    stryt = x['channel_title'].lower().replace(" ", "")
#     yt_musicdata_df['Artist'][index] = 
#     print(x['Channel Title'], " | ", stryt)
    
    for y in spotify_df['artists'].unique():
        str = y.lower().replace(" ", "")
        #print(y, " | ", str)
        if str in stryt:
            yt_merged2_df['artist'][index] = y


In [ ]:
yt_merged2_df.head(3)

In [ ]:
# Filter for only the Spotify's 2018 Top 100 Artists that also made it
# on YouTube's Top Trending Videos in 2018 in the Music Category 

yt_merged2_df = yt_merged2_df.loc[yt_merged2_df['artist'] != '']
yt_merged2_df = yt_merged2_df.reset_index(drop=True)

In [ ]:
# Please view the "artist" column next to teh channel title
yt_merged2_df

In [ ]:
# Extract Count and store in results
results = yt_merged2_df.groupby('artist').count()['channel_title'].sort_values(ascending=False)

# Store results into a dataframe
results_df = pd.DataFrame(results)

# Rename Columns
results_df = results_df.rename(columns={"channel_title":"top_trending_YT_videos_ct"})


In [ ]:
results_df